In [6]:
import sys
import os
from pathlib import Path

# Get absolute path to project root
project_root = Path(os.getcwd()).resolve()
if project_root.name != "ncg87-blockchain_tracker":
    project_root = project_root.parent  # Adjust if running from a subdirectory

sys.path.append(str(project_root))

# Add to Python path if not already there
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

from config import Settings

from database import MongoDatabase, MongoInsertOperations, MongoQueryOperations
from database import SQLDatabase, SQLInsertOperations, SQLQueryOperations


In [7]:
mongo_db = MongoDatabase()
mongo_query_ops = MongoQueryOperations(mongo_db)


In [8]:
query = mongo_query_ops.get_recent_transactions(network="Base", limit=100000)

In [9]:
query[0]


{'block_number': 25068607,
 'transaction_hash': '0xfde1efb93859014f23ff1905fe153cde3a4226c1a171217932ec592951db3272',
 'timestamp': 1736926561,
 'log_data': [{'event': 'Transfer',
   'parameters': {'from': {'value': '0xad4f8f6098232b250f2be2efd02248731246915d',
     'type': 'address',
     'indexed': True,
     'description': None},
    'to': {'value': '0x368ea6f645deb6d1a8692fe93c550d2834150865',
     'type': 'address',
     'indexed': True,
     'description': None},
    'value': {'value': 144976811691545014042624,
     'type': 'uint256',
     'indexed': False,
     'description': None}},
   'contract': '0xAC1Bd2486aAf3B5C0fc3Fd868558b082a531B2B4'},
  {'event': 'Transfer',
   'parameters': {'from': {'value': '0x368ea6f645deb6d1a8692fe93c550d2834150865',
     'type': 'address',
     'indexed': True,
     'description': None},
    'to': {'value': '0xad4f8f6098232b250f2be2efd02248731246915d',
     'type': 'address',
     'indexed': True,
     'description': None},
    'value': {'value':

In [10]:
len(query)


56183

In [11]:
import web3
import json
from database import SQLQueryOperations, SQLDatabase

sql_db = SQLDatabase()
sql_query_ops = SQLQueryOperations(sql_db)

In [2]:
contract_list = []
# need to check the broken abis
w3 = web3.Web3(web3.HTTPProvider(Settings.BASE_ENDPOINT))
for item in query:
    for log in item['log_data']:
        address = log['contract']

        abi = sql_query_ops.query_evm_contract_abi(network="Base", contract_address=address)
        if abi:
            abi_json = json.loads(abi['abi'])
            contract = w3.eth.contract(address, abi = abi_json)
            contract_list.append(contract)


NameError: name 'web3' is not defined

In [ ]:
ERC20_ABI = [
    {
        "constant": True,
        "inputs": [],
        "name": "name",
        "outputs": [{"name": "", "type": "string"}],
        "type": "function"
    },
    {
        "constant": True,
        "inputs": [],
        "name": "symbol",
        "outputs": [{"name": "", "type": "string"}],
        "type": "function"
    },
    {
        "constant": True,
        "inputs": [],
        "name": "decimals",
        "outputs": [{"name": "", "type": "uint8"}],
        "type": "function"
    }
]

In [1]:
contract_info = []
for i, contract in enumerate(contract_list):
    try:
        token_0 = contract.functions.token0().call()
        token_1 = contract.functions.token1().call()
        token0_contract = w3.eth.contract(address=token_0, abi=ERC20_ABI)
        token1_contract = w3.eth.contract(address=token_1, abi=ERC20_ABI)
        contract_factory = contract.functions.factory().call()
        contract_fee = contract.functions.fee().call()
        contract_token0 = token0_contract.functions.name().call()
        contract_token1 = token1_contract.functions.name().call()
        contract_info.append((
            contract.address,
            contract_factory,
            contract_fee,
            contract_token0,
            contract_token1
        ))
        print(i, contract_factory)
        print(i, contract_fee)
        print(i, contract_token0)
        print(i, contract_token1)
    except Exception as e:
        print(i, e)
        continue

NameError: name 'contract_list' is not defined

In [25]:
from analysis import PatternDetector


In [26]:
pattern_detector = PatternDetector()
for transaction in query:
    pattern_detector.process_transaction(transaction)



In [30]:
common_patterns = pattern_detector.get_patterns(min_frequency=100)


In [31]:
common_patterns


[{'sequence': [('0xde6533737cb4a37332b7bbb233ef5d18d898bcc2', 'Transfer'),
   ('0xde6533737cb4a37332b7bbb233ef5d18d898bcc2', 'Approval'),
   ('0xde6533737cb4a37332b7bbb233ef5d18d898bcc2', 'Approval'),
   ('0xde6533737cb4a37332b7bbb233ef5d18d898bcc2', 'Transfer'),
   ('0xde6533737cb4a37332b7bbb233ef5d18d898bcc2', 'Approval'),
   ('0x4200000000000000000000000000000000000006', 'Transfer'),
   ('0xcf60f60d60aa94bdc2728dbcfc7b2b36bd1079bc', 'Sync'),
   ('0xcf60f60d60aa94bdc2728dbcfc7b2b36bd1079bc', 'Swap'),
   ('0x4200000000000000000000000000000000000006', 'Withdrawal')],
  'frequency': 2781,
  'sample_tx': ['0xe0acf3894a1929ef9b2973ac64c518d6dfe758faa7893a514978dde41b1082df',
   '0x3e9717c9658595cf36cbf1edafa253e4044ec9753feb9e38569b25127090fd30',
   '0x3fbb33633cddd736576f26728d21d6d975b2fe9628da68c6d3c6876081ae664a',
   '0xb403b56a8349f1a5bac57b6aa4b0db6dab5f0759451170344706ea80befb67af',
   '0x5d17428a4c636fa45f6da39fb86ea5fe739c2cb0c89e63953e8bdb624f17e740']},
 {'sequence': [('0xde6533

In [32]:
len(common_patterns)


67

In [9]:
# In your notebook
pattern_detector = PatternDetector()
for transaction in query:
    pattern_detector.process_transaction(transaction)

analysis = pattern_detector.get_analysis(min_frequency=5)

# Print contract categories
print("\nContract Categories:")
for category, count in analysis['contract_analysis']['contract_categories'].items():
    print(f"{category}: {count} contracts")

# Print most active contracts
print("\nMost Active Contracts:")
for contract in analysis['contract_analysis']['most_active_contracts']:
    print(f"\nAddress: {contract['address']}")
    print(f"Category: {contract['category']}")
    print(f"Interactions: {contract['interaction_count']}")
    print(f"Events: {', '.join(contract['event_types'])}")

# Print interesting patterns
print("\nInteresting Contract Interaction Patterns:")
for pattern in analysis['contract_analysis']['interesting_patterns']:
    print(f"\nMain Contract: {pattern['main_contract']['address']} ({pattern['main_contract']['category']})")
    print("Interacts with:")
    for related in pattern['related_contracts']:
        print(f"  - {related['address']} ({related['category']})")


Contract Categories:
DEX: 1643 contracts
NFT: 1190 contracts
Bridge: 21 contracts
Lending: 126 contracts
Unknown: 269 contracts

Most Active Contracts:

Address: 0x4200000000000000000000000000000000000006
Category: DEX
Interactions: 66616
Events: Approval, Transfer, MintClaimed, Mint, Withdrawal, RewardsDeposit, Sync, Deposit, Collect, Swap

Address: 0xfec698608fe71ff77c2b2a8e7fc5253f84b1f8dc
Category: NFT
Interactions: 39860
Events: Transfer

Address: 0x827922686190790b37229fd06084350e74485b72
Category: DEX
Interactions: 17822
Events: IncreaseLiquidity, Approval, Burn, Transfer, DecreaseLiquidity, TokenReturned, MetadataUpdate, Collect, Swap, ApprovalForAll

Address: 0xde6533737cb4a37332b7bbb233ef5d18d898bcc2
Category: NFT
Interactions: 17531
Events: Approval, Transfer

Address: 0x833589fcd6edb6e08f4c7c32d4f71b54bda02913
Category: Bridge
Interactions: 14609
Events: Approval, Unknown, Fees, Transfer, Withdrawal, Exchange, Sync, Deposit, Swap

Address: 0xcf60f60d60aa94bdc2728dbcfc7b2b3

In [11]:
query[0]


{'block_number': 25024669,
 'transaction_hash': '0x89b713569bf63ce00816fb2baf8f73c8178021962a6d1d87ff9b28ea4db92550',
 'timestamp': 1736838685,
 'log_data': [{'event': 'Deposit',
   'parameters': {'dst': {'value': '0x4752ba5dbc23f44d87826276bf6fd6b1c372ad24',
     'type': 'address',
     'indexed': True,
     'description': None},
    'wad': {'value': 54360000000000000,
     'type': 'uint256',
     'indexed': False,
     'description': None}},
   'contract': '0x4200000000000000000000000000000000000006'},
  {'event': 'Transfer',
   'parameters': {'from': {'value': '0x4752ba5dbc23f44d87826276bf6fd6b1c372ad24',
     'type': 'address',
     'indexed': True,
     'description': None},
    'to': {'value': '0xcf60f60d60aa94bdc2728dbcfc7b2b36bd1079bc',
     'type': 'address',
     'indexed': True,
     'description': None},
    'value': {'value': 54360000000000000,
     'type': 'uint256',
     'indexed': False,
     'description': None}},
   'contract': '0x4200000000000000000000000000000000000

In [18]:
import psycopg2

conn = psycopg2.connect(
    host='dex-project-db.cvqeaeicw5w1.us-east-1.rds.amazonaws.com',
    port=5432,
    database='dex_transactions',
    user='nicko',
    password='rumpie123'
)

cursor = conn.cursor()

cursor.execute("SELECT * FROM token_metadata WHERE id = '0x4200000000000000000000000000000000000006';")

result = cursor.fetchall()

result



[('0x4200000000000000000000000000000000000006',
  'unknown',
  'unknown',
  datetime.datetime(2025, 1, 4, 5, 7, 19, 865304))]

In [6]:
cursor.execute("SELECT * FROM token_metadata;")

result1 = cursor.fetchall()



1

In [7]:
result1

[('0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2',
  'WETH',
  'Wrapped Ether',
  datetime.datetime(2025, 1, 4, 4, 57, 42, 798683)),
 ('0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48',
  'USDC',
  'USD Coin',
  datetime.datetime(2025, 1, 4, 4, 57, 42, 798683)),
 ('0xdac17f958d2ee523a2206206994597c13d831ec7',
  'USDT',
  'Tether USD',
  datetime.datetime(2025, 1, 4, 4, 57, 42, 798683)),
 ('0x2260fac5e5542a773aa44fbcfedf7c193bc2c599',
  'WBTC',
  'Wrapped BTC',
  datetime.datetime(2025, 1, 4, 4, 57, 42, 798683)),
 ('0x6b175474e89094c44da98b954eedeac495271d0f',
  'DAI',
  'Dai Stablecoin',
  datetime.datetime(2025, 1, 4, 4, 57, 42, 798683)),
 ('0x7f39c581f595b53c5cb19bd0b3f8da6c935e2ca0',
  'wstETH',
  'Wrapped liquid staked Ether 2.0',
  datetime.datetime(2025, 1, 4, 4, 57, 42, 798683)),
 ('0x4c9edd5852cd905f086c759e8383e09bff1e68b3',
  'USDe',
  'USDe',
  datetime.datetime(2025, 1, 4, 4, 57, 42, 798683)),
 ('0x514910771af9ca656af840dff83e8264ecf986ca',
  'LINK',
  'ChainLink Token',
  datetime